In [1]:
import sqlite3
import pandas as pd
import random
import re
random.seed(1)
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup

In [2]:
db_path = '/Users/boyaliu/Documents/UCD/STA 208/final project/data/so-dump.db'
conn = sqlite3.connect(db_path)

In [3]:
query = "SELECT * FROM posts"

In [4]:
post_df = pd.read_sql_query(query, conn)

In [5]:
post_df.drop(['LastEditorDisplayName','CommunityOwnedDate','LastEditorUserId','LastEditDate',
             'LastActivityDate'],axis=1,inplace=True)

In [6]:
qpost_df = post_df[post_df.PostTypeId==1]

In [7]:
qpost_df.shape

(75067, 16)

In [43]:
# create a list for stopwords including uppercase stopwords, lowercase stopwords and capitalized stopwords 
stop_words = stopwords.words('english') + [x.upper() for x in stopwords.words('english')] + [x.capitalize() for x in stopwords.words('english')]

In [92]:
# clean the data set

Code = []

def exact_from_html(row):
    soup = BeautifulSoup(row, "html.parser")

    #remove code
    if soup.code is not None:
        code = soup.code.extract()
        Code.append(code)
    else:
        Code.append('NA')
        
    raw = soup.get_text()
    #remove link
    raw_no_link = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", '', raw)
    #remove email
    no_link_email = re.sub(r'[\w\.-]+@[\w\.-]+[\.][com|org|ch|uk]{2,3}', "", raw_no_link)
    #remove fomula
    no_link_email_fomula= re.sub('(\$.+?\$)', "", no_link_email, flags=re.IGNORECASE)
    #remove whitespace
    tab_text = '\t\n\r\x0b\x0c'
    raw = "".join([ch for ch in no_link_email_fomula if ch not in set(tab_text)])
    return(raw)

def remove_stop(row):
    raw = " ".join([word for word in word_tokenize(row) if word not in stop_words])
    return(raw)
def remove_punc(row):
    raw = "".join([ch for ch in row if ch not in set(string.punctuation)])
    return(raw)




In [94]:
# get the body without removing stopwords and punctuation 
body_wo_sto_pun = qpost_df['Body'].map(lambda i: exact_from_html(i))

In [95]:
# get the title without removing stopwords and punctuation 
title_wo_sto_pun = qpost_df['Title'].map(lambda i: exact_from_html(i))

In [96]:
# get the body for sentimental analysis, did not remove punctuation 
body_senti = body_wo_sto_pun.map(lambda i: remove_stop(i))

In [97]:
# get the title for sentimental analysis, did not remove punctuation 
title_senti = title_wo_sto_pun.map(lambda i: remove_stop(i))

In [98]:
# get the clean title
title_clean = title_senti.map(lambda i: remove_punc(i))

In [99]:
# get the clean body
Body_clean = body_senti.map(lambda i: remove_punc(i))

In [100]:
# get the body and title without removing stopwords and punctuation 
body_title_wo = body_wo_sto_pun + title_wo_sto_pun

In [102]:
# count the space
space_count = body_title_wo.map(lambda i: i.count(' '))

In [111]:
len(space_count)

75067

In [103]:
# combine title and body together
body_title = Body_clean + title_clean

In [104]:
def case_count(row):
    # count lower_case percentage & upper_case percentage
    up = []
    row_nw = row.replace(" ", "")
    for c in row:
        if c.isupper():
            up.append(1)
            
    upper_per = sum(up)/len(row_nw)    
    lower_per = 1 - upper_per
    return((lower_per, upper_per))

In [105]:
body_title[1]

'many different statistical methods  assumption normality    normality  know normality normality '

In [106]:
# count the case 
c_c = body_title.map(lambda i: case_count(i))
upper_per = c_c.map(lambda i: i[1])
lower_per = c_c.map(lambda i: i[0])



In [107]:
len(upper_per)

75067

In [108]:
upper_per.head()

0    0.027397
1    0.000000
2    0.034884
3    0.019231
5    0.038750
dtype: float64

In [109]:
lower_per.head()

0    0.972603
1    1.000000
2    0.965116
3    0.980769
5    0.961250
dtype: float64

In [112]:
# count the url
def url_count(row):
    url = []
    soup = BeautifulSoup(row, "html.parser")
    raw = BeautifulSoup(row,"html.parser").get_text()
    for a in soup.find_all('a', href=True):
        url.append(a['href'])
    url_re = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', raw)
    for i in url_re:
        url.append(i)
    return(len(set(url)))

In [113]:
# count the number of url
url_count = qpost_df.Body.map(lambda i: url_count(i))

In [114]:
url_count.head()

0    0
1    0
2    0
3    0
5    4
Name: Body, dtype: int64

In [115]:
len(url_count)

75067

In [116]:
# calculate the body length
body_len = qpost_df.Body.str.len()
body_len.head()

0      91
1     146
2     206
3     497
5    1628
Name: Body, dtype: int64

In [117]:
len(body_len)

75067

In [118]:
# calculate the title length
title_len = qpost_df.Title.str.len()
title_len.head()


0    29
1    18
2    65
3    58
5    50
Name: Title, dtype: int64

In [119]:
# count the number of tags
tag_count = qpost_df.Tags.map(lambda i: i.count('<'))
tag_count.head()

0    3
1    2
2    2
3    2
5    1
Name: Tags, dtype: int64

In [120]:
# check the similarity between title and body
simil = []
for i in range(0,len(Body_clean)):
    aa = []
    aa.append(Body_clean.tolist()[i])
    aa.append(title_clean.tolist()[i])
    tfidf = TfidfVectorizer().fit_transform(aa)
    pairwise_similarity = tfidf * tfidf.T
    simil.append(pairwise_similarity[0,1])

In [121]:
len(simil)

75067

In [124]:
# sentimental analysis for title
sen_score_title = title_senti.map(lambda i: nltk.sentiment.vader.SentimentIntensityAnalyzer().polarity_scores(i)['compound'])

In [125]:
# sentimental analysis for body
sen_score_body = body_senti.map(lambda i: nltk.sentiment.vader.SentimentIntensityAnalyzer().polarity_scores(i)['compound'])

In [126]:
# integrate all StackOverflow Metrics features
qpost_df['BodyLen'] = body_len
qpost_df['LowerCasePer'] = lower_per
qpost_df['UpperCasePer'] = upper_per
qpost_df['SpaceCnt'] = space_count
qpost_df['TagCnt'] = tag_count
qpost_df['BodySenScore'] = sen_score_body
qpost_df['TitleSenScore'] = sen_score_title
qpost_df['TitleLen'] = title_len
qpost_df['UrlCnt'] = url_count
qpost_df['CleanBody'] = Body_clean
qpost_df['CleanTitle'] = title_clean
qpost_df['SentiBody'] = body_senti
qpost_df['SentiTitle'] = title_senti
qpost_df['Ntprocess_stop_pun_Body'] = body_wo_sto_pun
qpost_df['Ntprocess_stop_pun_Title'] = title_wo_sto_pun

/Users/boyaliu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/boyaliu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/boyaliu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [127]:
qpost_df.shape

(75067, 31)

In [128]:
target_cols = ['Id','ClosedDate','Score','BodyLen','LowerCasePer','UpperCasePer','SpaceCnt','TagCnt', 'BodySenScore',
             'TitleSenScore',  'TitleLen','UrlCnt']

In [129]:
stackoverflow_metric_df = qpost_df[target_cols]


In [137]:
stackoverflow_metric_df.head(10)


,Id,ClosedDate,Score,BodyLen,LowerCasePer,UpperCasePer,SpaceCnt,TagCnt,BodySenScore,TitleSenScore,TitleLen,UrlCnt
0,1,None,31,91,0.972603,0.027397,15,3,0.0000,0.0000,29,0
1,2,None,26,146,1.000000,0.000000,25,2,0.0000,0.0000,18,0
2,3,None,63,206,0.965116,0.034884,36,2,0.8126,0.4767,65,0
3,4,None,15,497,0.980769,0.019231,90,2,0.5574,0.2732,58,0
5,6,None,226,1628,0.961250,0.038750,204,1,0.8917,0.0000,50,4
6,7,None,87,721,0.994071,0.005929,109,4,0.8442,0.4404,38,0
7,8,2010-07-19T20:19:46.577,0,121,0.978495,0.021505,32,1,-0.3802,-0.1027,62,0
9,10,None,32,171,0.971831,0.028169,42,5,0.0000,0.0000,79,0
10,11,None,3,738,0.986957,0.013043,86,2,0.7550,0.0000,37,1
16,17,None,10,573,0.942197,0.057803,103,3,0.3182,0.0000,38,0


In [134]:
stackoverflow_metric_df.to_pickle('/Users/boyaliu/Documents/UCD/STA 208/final project/stackoverflow_metric_df.pkl')


In [135]:
qpost_df.to_pickle('/Users/boyaliu/Documents/UCD/STA 208/final project/qpost_df_new.pkl')
